### [PCCP 기출문제] 1번 / 붕대 감기

In [113]:
def solution(bandage, health, attacks):
    max_health = health
    success_cnt = 0
    attack_dict = {atk_time: dmg for atk_time, dmg in attacks}
    for time in range(0, attacks[-1][0] + 1):
        if time in attack_dict:
            success_cnt = 0
            damage = attack_dict[time]
            health -= damage
            if health <= 0:
                return -1         
        else:
            health = min(health + bandage[1], max_health)
            success_cnt += 1
            if success_cnt == bandage[0]:
                success_cnt = 0
                health = min(health + bandage[2], max_health)  
    return -1 if health <= 0 else health


### [PCCP 기출문제] 2번 / 석유 시추

In [114]:
def solution(land):
    def dfs_iterative(matrix, i, j):
        stack = [(i, j)]
        group = set()  # 현재 그룹의 열을 저장할 집합
        count = 0  # 현재 그룹의 크기
        while stack:
            x, y = stack.pop()
            if x < 0 or x >= len(matrix) or y < 0 or y >= len(matrix[0]) or matrix[x][y] == 0 or visited[x][y]:
                continue
            visited[x][y] = True
            count += 1
            group.add(y)
            # 상, 하, 좌, 우 탐색
            stack.extend([(x+1, y), (x-1, y), (x, y+1), (x, y-1)])
        return group, count

    visited = [[False for _ in row] for row in land]
    col_sizes = [0] * len(land[0])
    for i in range(len(land)):
        for j in range(len(land[0])):
            if land[i][j] == 1 and not visited[i][j]:
                group, count = dfs_iterative(land, i, j)
                for col in group:
                    col_sizes[col] += count

    return max(col_sizes)

### [PCCP 기출문제] 3번 / 아날로그 시계

In [70]:
def convert_to_second(h, m, s):
    return 3600 * h + 60 * m + s

def solution(h1, m1, s1, h2, m2, s2):
    start_time, end_time = convert_to_second(h1, m1, s1), convert_to_second(h2, m2, s2)
    ring = {}

    # 초기화
    for t in range(convert_to_second(24, 0, 0)):
        ring[t] = {"cur": 0, "after": 0}

    # 이전 시각의 초침, 분침, 시침의 각도
    prev_second, prev_minute, prev_hour = 0, 0, 0

    for t in range(1, convert_to_second(24, 0, 0)):
        after_count, cur_count = 0, 0

        # 현재 시각의 초침, 분침, 시침의 각도
        cur_second, cur_minute, cur_hour = 6 * t % 360, (t / 10) % 360, (t / 120) % 360

        # 첫 번째 경우: 초침이 분침을 따라 잡은 경우
        if prev_second < prev_minute and cur_second > cur_minute:
            after_count += 1
        # 두 번째 경우: 초침이 분침을 따라 잡았지만 현재의 초침의 위치가 0인 경우
        elif prev_second < prev_minute and cur_second == 0:
            after_count += 1
        # 세 번째 경우: 초침이 분침과 현재 시각에 위치가 정확히 같은 경우
        elif cur_second == cur_minute:
            cur_count += 1

        # 첫 번째 경우: 초침이 시침을 따라 잡은 경우
        if prev_second < prev_hour and cur_second > cur_hour:
            after_count += 1
        # 두 번째 경우: 초침이 시침을 따라 잡았지만 현재의 초침의 위치가 0인 경우
        elif prev_second < prev_hour and cur_second == 0:
            after_count += 1
        # 세 번째 경우: 초침이 시침과 현재 시각에 위치가 정확히 같은 경우
        elif cur_second == cur_hour:
            cur_count += 1

        # t에 cur과 after를 각각 반영, 12시 0분 0초에는 반영하지 않음
        if t != 3600 * 12:
            ring[t]["cur"] += cur_count
            ring[t - 1]["after"] += after_count

        # prev 업데이트
        prev_second, prev_minute, prev_hour = cur_second, cur_minute, cur_hour

    # 0시 0분 0초와 12시 0분 0초는 한 번만 울림
    ring[0] = {"cur": 1, "after": 0}
    ring[3600 * 12] = {"cur": 1, "after": 0}

    ret = 0

    for time in range(start_time, end_time):
        ret += ring[time]["after"]
        ret += ring[time]["cur"]

    # endTime에는 after를 반영하지 않음
    ret += ring[end_time]["cur"]
    return ret


In [79]:
def solution(h1, m1, s1, h2, m2, s2):
    answer = 0
    start = h1*3600+m1*60+s1
    end   = h2*3600+m2*60+s2
    answer+=int(end/3600*59)
    print(answer)
    answer-=start//(3600/59)
    answer+=end//(360*120/719)
    answer-=start//(360*120/719)
    if start<=12*60*60<=end:answer-=1
    if start==0 or start==12*60*60:answer+=1

    return answer

### [PCCP 기출문제] 4번 / 수레 움직이기

In [36]:
from collections import deque

def msbfs(grid, start1, end1, start2, end2):
    queue = deque([((start1[0], start1[1], start2[0], start2[1]),0,0)])  # 두 점의 위치와 각각의 거리
    visited = set([(start1, start2)])
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # 상하좌우 이동

    blank_exist = False
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == 0: blank_exist = True
    if blank_exist == False : return 0

    while queue:
        (x1, y1, x2, y2), dist1, dist2 = queue.popleft()
          
        if (x1, y1) == end1 and (x2, y2) == end2:
            return max(dist1, dist2) # 두 점 모두 목적지에 도달

        for dx1, dy1 in directions:
            new_x1, new_y1 = x1, y1  # 현재 위치를 기본값으로 설정
            if (x1, y1) != end1:
                new_x1, new_y1 = x1 + dx1, y1 + dy1
                if not (0 <= new_x1 < len(grid) and 0 <= new_y1 < len(grid[0]) and grid[new_x1][new_y1] != 5):
                    continue

            for dx2, dy2 in directions:
                new_x2, new_y2 = x2, y2  # 현재 위치를 기본값으로 설정
                if (x2, y2) != end2:
                    new_x2, new_y2 = x2 + dx2, y2 + dy2
                    if not (0 <= new_x2 < len(grid) and 0 <= new_y2 < len(grid[0]) and grid[new_x2][new_y2] != 5):
                        continue

                # 수레끼리 위치를 교환하는 경우 배제
                if (new_x1, new_y1) == (x2, y2) and (new_x2, new_y2) == (x1, y1):
                    continue

                if (new_x1, new_y1) != (new_x2, new_y2) and ((new_x1, new_y1, new_x2, new_y2) not in visited):
                    visited.add((new_x1, new_y1, new_x2, new_y2))
                    queue.append(((new_x1, new_y1, new_x2, new_y2), dist1 + int((x1, y1) != end1), dist2 + int((x2, y2) != end2)))

    return 0  # 경로가 없는 경우

def solution(maze):
    coord_dict = {"red":{},"blue":{}}
    for x in range(len(maze)):
        for y in range(len(maze[x])):
            # 빨간 수레의 시작 위치
            if maze[x][y] == 1: coord_dict["red"]["start"] = (x,y)
            # 파란 수레의 시작 위치
            if maze[x][y] == 2: coord_dict["blue"]["start"] = (x,y)
            # 빨간 수레의 도착 위치
            if maze[x][y] == 3: coord_dict["red"]["end"] = (x,y)
            # 파란 수레의 도착 위치
            if maze[x][y] == 4: coord_dict["blue"]["end"] = (x,y)
    
    turn_cnt = msbfs(grid = maze, 
                      start1 = coord_dict["red"]["start"], 
                      end1 = coord_dict["red"]["end"], 
                      start2 = coord_dict["blue"]["start"],
                      end2 = coord_dict["blue"]["end"])
            
    return turn_cnt

maze1 = [[1, 4], [0, 0], [2, 3]] #3
maze2 = [[1, 0, 2], [0, 0, 0], [5, 0 ,5], [4, 0, 3]] #7
maze3 = [[1, 5], [2, 5], [4, 5], [3, 5]] #0
maze4 = [[4, 1, 2, 3]] #0
maze5 = [[4, 3, 0, 0], [5, 5, 5, 0], [1, 0, 0, 0], [2, 0, 0, 0]] #9
maze6 = [[2,1,0,0], [0,0,5,0], [0,0,0,0], [3,5,4,0]] #5
print(solution(maze1))
print(solution(maze2))
print(solution(maze3))
print(solution(maze4))
print(solution(maze5))
print(solution(maze6))

3
7
0
0
9
5


In [44]:
class Move:
    def __init__(self, maze):
        self.maze = maze
        self.n, self.m = len(maze), len(maze[0])
        self.red, self.blue, self.target_red, self.target_blue = self.find_positions()
        self.directions = [(0, -1), (0, 1), (1, 0), (-1, 0)]
        self.global_minimum = float('inf')

    def find_positions(self):
        positions = {}
        for row in range(self.n):
            for col in range(self.m):
                if self.maze[row][col] in [1, 2, 3, 4]:
                    positions[self.maze[row][col]] = (row, col)
        return positions[1], positions[2], positions[3], positions[4]

    def validation(self, cur, visited, target):
        if cur == target:
            return [target]

        x, y = cur
        possible_moves = []
        for dx, dy in self.directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.n and 0 <= ny < self.m and (nx, ny) not in visited and self.maze[nx][ny] != 5:
                possible_moves.append((nx, ny))

        return possible_moves

    def brute_force(self, red, blue, turn, red_visited, blue_visited):
        if red == self.target_red and blue == self.target_blue:
            self.global_minimum = min(self.global_minimum, turn)
            return

        red_possible_moves = self.validation(red, red_visited, self.target_red)
        blue_possible_moves = self.validation(blue, blue_visited, self.target_blue)

        for rp in red_possible_moves:
            for bp in blue_possible_moves:
                if rp != bp and (rp, bp) != (blue, red):
                    self.brute_force(rp, bp, turn + 1, red_visited | {rp}, blue_visited | {bp})

    def solve(self):
        self.brute_force(self.red, self.blue, 0, {self.red}, {self.blue})
        return self.global_minimum if self.global_minimum != float('inf') else 0


def solution(maze):
    path = Move(maze)
    return path.solve()

maze1 = [[1, 4], [0, 0], [2, 3]] #3
maze2 = [[1, 0, 2], [0, 0, 0], [5, 0 ,5], [4, 0, 3]] #7
maze3 = [[1, 5], [2, 5], [4, 5], [3, 5]] #0
maze4 = [[4, 1, 2, 3]] #0
maze5 = [[4, 3, 0, 0], [5, 5, 5, 0], [1, 0, 0, 0], [2, 0, 0, 0]] #9

print(solution(maze1))
print(solution(maze2))
print(solution(maze3))
print(solution(maze4))
print(solution(maze5))

3
7
0
0
9
